In [2]:
import pandas as pd
import joblib
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
from nb_model import mulitnomialNB

df = pd.read_csv('sms.tsv', sep='\t', header=None, names=['l', 'message'])
df['message'] = df['message'].fillna('')

x = df['message']
y = df['l']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=23)

vectorizer = CountVectorizer()
x_v_train = vectorizer.fit_transform(x_train)
x_v_test = vectorizer.transform(x_test)

y_train = y_train.values
y_test = y_test.values

model = mulitnomialNB()
model.fit(x_v_train, y_train)

y_pred = model.predict(x_v_test)

print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

joblib.dump(vectorizer, "vectorizer.pkl")
joblib.dump(model, "model.pkl")

print("Model and vectorizer saved!")


0.9901345291479821
[[968   2]
 [  9 136]]
Model and vectorizer saved!
